In [ ]:
import pandas as pd
import numpy as np
import re
from Bio import SeqIO
from itertools import combinations

metadata = pd.read_csv("../wilmurt_meta.csv")

strains = ['Assinica (Hatchery)', 'Domestic (Hatchery)',
       'Horn (Hatchery)', 'Little Tupper (Hatchery)', 
           'Temiscamie (Hatchery)', 'Windfall (Hatchery)']
for comb in combinations(strains, 4):
    combo = comb[0][0]+comb[1][0]+comb[2][0]+comb[3][0]
    hatchery = (metadata["Novogene_ID"][metadata["Source"].isin(list(comb))]
            [metadata["Source"].str.contains("Hatchery")]
            [~metadata["Novogene_ID"].str.contains("TxD")]).tolist()
    for seq_record in SeqIO.parse("../ingroup_and_outgroup_circularized.msa", "fasta"):
        if str(seq_record.id) in hatchery:
            for i in list(range(len(seq_record.seq) - 450))[0::5]:
                with open("../gene_windows/" + str(i) + "_" + combo + ".fasta","a") as f:
                   f.write(">" + str(seq_record.id) + "\n")
                   f.write(str(seq_record.seq[i:i+450].ungap("-")) + "\n")

/storage/group/ibb3/default/.conda/envs/masters/lib/python3.10/site-packages/Bio/Seq.py:2096: BiopythonDeprecationWarning: myseq.ungap(gap) is deprecated; please use myseq.replace(gap, "") instead.
  warnings.warn(


In [ ]:
%%bash
source ~/.bashrc
conda activate ipyrad
for i in $( ls ../gene_windows/*_????.fasta | sed "s/.fasta//g" | cut -d"/" -f3 ); do
    vsearch --derep_fulllength ../gene_windows/${i}.fasta \
        --output ../gene_windows/${i}.derep.fasta \
        --sizeout --threads 8 \
        --uc ../gene_windows/${i}.derep.uc \
        --fasta_width 0
done